# Use Multiplanr design, for Ansys Q3D simulation using pyaedt renderer to get the capacitance matrixes. 

**DISCLAIMER:** For now, this might only work with the full version (paid) of Ansys and not the student version. This issue is fixable and requires more testing. Please run the notebook and test it out on student license, if you're interested to make a contribution!

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Use LayerStack file to identify z-coordinate for layers; also denote the material and fill information.¶
## The file format is a csv file.

In [3]:
import sys

In [4]:
from qiskit_metal.renderers.renderer_ansys_pyaedt.pyaedt_base import QPyaedt
from qiskit_metal.designs.design_multiplanar import MultiPlanar
from qiskit_metal.renderers.renderer_ansys_pyaedt.q3d_renderer_aedt import QQ3DPyaedt
from qiskit_metal.renderers.renderer_ansys_pyaedt.hfss_renderer_drivenmodal_aedt import QHFSSDrivenmodalPyaedt
from qiskit_metal.renderers.renderer_ansys_pyaedt.hfss_renderer_eigenmode_aedt import QHFSSEigenmodePyaedt

import numpy as np
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

## Use LayerStack file to identify z-coordinate for layers; also denote the material and fill information.
### The file format is a csv file.

In [5]:
# User needs to update next line to their own installation.
ls_file_path =  r"..\..\..\resources\layer_stack_data_example_1.csv"

multiplanar_design = MultiPlanar(metadata={},
                                 overwrite_enabled=True,
                                 layer_stack_filename=ls_file_path)

## Multi-Planar design has default of chip named main.  If there is more than one, add information to multiplanar design.

In [6]:
multiplanar_design._chips['qubit_chip'] = Dict()
multiplanar_design._chips.qubit_chip.size = Dict(
    center_x='0.0mm',
    center_y='0.0mm',
    size_x='4mm',
    size_y='7mm',
)


# Over-ride the default values size for chip=='main'.
multiplanar_design.chips.main.size['size_x'] = '2mm'
multiplanar_design.chips.main.size['size_y'] = '2mm'

In [7]:
multiplanar_design.chips

{'main': {'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'size_x': '2mm',
   'size_y': '2mm'}},
 'qubit_chip': {'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'size_x': '4mm',
   'size_y': '7mm'}}}

In [8]:
#Remember, the layers may not always be unique, since there can be multiple datatypes.
ls_unique = multiplanar_design.ls.is_layer_data_unique()

## Start the Qiskit Metal GUI by using multi-planar design.

In [9]:
"""
Note: If you get an error to install a font, then in your terminal command line, install qdarkstyle:
pip install qdarkstyle


Then re-run this cell.
"""


gui = MetalGUI(multiplanar_design)

## Add all components to design.  

### Note the layer number and chip names are used when adding components to design. 

In [10]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.qubits.transmon_concentric import TransmonConcentric

# As precaution, remove any components already previously to design.
multiplanar_design.delete_all_components()


qubit_cpw_otg_layer = 2
qubit_chip_name = 'qubit_chip'


q1 = TransmonPocket(
    multiplanar_design,
    'Q1',
    options=dict(
        pad_width='425 um',
        pocket_height='650um',
        layer=qubit_cpw_otg_layer,
        chip=qubit_chip_name,
        #orientation='44',
        orientation='0',
        connection_pads=dict(
            readout=dict(loc_W=+1, loc_H=+1, pad_width='200um'))))

otg = OpenToGround(multiplanar_design,
                   'open_to_ground',
                   options=dict(pos_x='1.75mm',
                                pos_y='0um',
                                orientation='0',
                                layer=qubit_cpw_otg_layer,
                                chip=qubit_chip_name))

route_meaander = RouteMeander(
    multiplanar_design, 'readout',
    Dict(
        layer=qubit_cpw_otg_layer,
        chip=qubit_chip_name,
        total_length='3 mm',
        hfss_wire_bonds=True,
        fillet='90 um',
        lead=dict(start_straight='100um'),
        pin_inputs=Dict(start_pin=Dict(component='Q1', pin='readout'),
                        end_pin=Dict(component='open_to_ground', pin='open')),
    ))

##############################################################################
optionsQ = dict(
    pad_width='425 um',
    pocket_height='650um',
    layer=qubit_cpw_otg_layer,
    chip=qubit_chip_name,
    orientation='-44',
    #orientation='0',
    connection_pads=dict(  # Qbits defined to have 4 pins
        a=dict(loc_W=+1, loc_H=+1),
        b=dict(loc_W=-1, loc_H=+1, pad_height='30um'),
        c=dict(loc_W=+1, loc_H=-1, pad_width='200um'),
        d=dict(loc_W=-1, loc_H=-1, pad_height='50um')))

q2 = TransmonPocket(multiplanar_design,
                    'Q2',
                    options=dict(pos_x='-1.5mm', pos_y='+0.0mm', **optionsQ))

##############################################################################
concentric_options = dict(
    #chip='main',
    pos_x='400um',
    pos_y='900um',
    #layer='1',  # default is 1, this is just for example. This component does NOT have option for layer.
    pocket_w='1500um',  # transmon pocket width
    pocket_h='900um',  # transmon pocket height
)

# Create a new Concentric Transmon object with name 'Q1'
qc1 = TransmonConcentric(multiplanar_design,
                         'cc_qubit',
                         options=concentric_options)


gui.rebuild()
gui.autoscale()

# Start the Q3D renderer for Ansys by passing arguments to QQ3DPyaedt.
## The default will use default names for Ansys ProjectName and Ansys DesignName.

In [11]:
multi_q3d = QQ3DPyaedt(multiplanar_design)

# Example of passing names for project and design.
#multi_q3d_1 = QQ3DPyaedt(multiplanar_design, "q3d_project_1", "q3d_design_1")

In [12]:
"""Create a solution setup in Ansys Q3D. If user does not provide
        arguments, they will be obtained from QQ3DPyaedt.default_setup dict.

Args:
    name (str, optional): Name of solution setup. Defaults to None.
    AdaptiveFreq (float, optional): Adaptive frequency in GHz. Defaults to None.
    SaveFields (bool, optional): Whether or not to save fields. Defaults to None.
    Enabled (bool, optional): Whether or not setup is enabled. Defaults to None.
    MaxPass (int, optional): Maximum number of passes. Defaults to None.
    MinPass (int, optional): Minimum number of passes. Defaults to None.
    MinConvPass (int, optional): Minimum number of converged passes. Defaults to None.
    PerError (float, optional): Error tolerance as a percentage. Defaults to None.
    PerRefine (int, optional): Refinement as a percentage. Defaults to None.
    AutoIncreaseSolutionOrder (bool, optional): Whether or not to increase solution order automatically. Defaults to None.
    SolutionOrder (str, optional): Solution order. Defaults to None.
    Solver_Type (str, optional): Solver type. Defaults to None.
"""

QQ3DPyaedt.default_setup

{'name': 'QQ3DPyaedt_setup',
 'AdaptiveFreq': '5.0',
 'SaveFields': 'False',
 'Enabled': 'True',
 'MaxPass': '15',
 'MinPass': '2',
 'MinConvPass': '2',
 'PerError': '0.5',
 'PerRefine': '30',
 'AutoIncreaseSolutionOrder': 'True',
 'SolutionOrder': 'High',
 'Solver_Type': 'Iterative'}

In [13]:
# Use all default values.
multi_q3d.add_q3d_setup() 

# Reduce MaxPass when running notebook as an example.
multi_q3d.add_q3d_setup(name='fun_test', MaxPass=3)  

pyaedt INFO: using existing logger.


INFO:Global:using existing logger.


pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\Ansys Student\v222\Win64


INFO:Global:AEDT installation Path C:\Program Files\AnsysEM\Ansys Student\v222\Win64


pyaedt INFO: Launching AEDT with the gRPC plugin.


...

INFO:Global:Design Loaded


pyaedt INFO: Successfully loaded project materials !


INFO:Global:Successfully loaded project materials !


pyaedt INFO: Materials Loaded


INFO:Global:Materials Loaded


# Render the multi-planar design using the similar convention as planar-design to identify ports. 

In [14]:
multi_q3d.render_design(selection=['Q1'],
                        box_plus_buffer=True,
                        open_pins=[('Q1', 'readout')])


############# Below are samples of different rendering arguments.

#multi_q3d.render_design(selection=['Q1', 'Q2', 'cc_qubit'],
#                        box_plus_buffer=True,
#                        open_pins=[('Q1', 'readout'), ('Q2', 'a'), ('Q2', 'b'),
#                        ('Q2', 'd')])


#multi_q3d.render_design(selection=['mult_planar_readout'])

#multi_q3d.render_design(selection=['readout'])

#multi_q3d.render_design(selection=[], box_plus_buffer=False)

#multi_q3d.render_design(selection=['Q1', 'readout'], box_plus_buffer=True)

#multi_q3d.render_design(selection=[], box_plus_buffer=False)

pyaedt INFO: Project QQ3DPyaedt_project set to active.


INFO:Global:Project QQ3DPyaedt_project set to active.


pyaedt INFO: Aedt Objects initialized


INFO:Global:Aedt Objects initialized


pyaedt INFO: Variable Manager initialized


...

INFO:Global:Design Loaded


pyaedt INFO: Successfully loaded project materials !


INFO:Global:Successfully loaded project materials !


pyaedt INFO: Materials Loaded


INFO:Global:Materials Loaded


## Select the name to analyze.  Presently, we have two to choose from.  The default=='QQ3DPyaedt_setup' and 'fun_test'.

In [15]:
multi_q3d.analyze_setup('fun_test')

pyaedt INFO: Project QQ3DPyaedt_project set to active.


INFO:Global:Project QQ3DPyaedt_project set to active.


pyaedt INFO: Aedt Objects initialized


INFO:Global:Aedt Objects initialized


pyaedt INFO: Variable Manager initialized


...

INFO:Global:5 Nets have been identified: layer_1_datatype_0_plane, layer_2_datatype_0_plane, readout_connector_pad_Q1, pad_top_Q1, pad_bot_Q1


pyaedt INFO: Solving design setup fun_test


INFO:Global:Solving design setup fun_test


pyaedt INFO: Design setup fun_test solved correctly in 0.0h 0.0m 9.0s


INFO:Global:Design setup fun_test solved correctly in 0.0h 0.0m 9.0s


## Get the result of analyze_setup.
### You can get the capacitance matrix in pandas dataframes format, for the AdaptiveFreq in GHz which was 5.0 in our example.

In [16]:
cap_data = multi_q3d.get_capacitance_all_passes('fun_test')

pyaedt INFO: Solution Data Correctly Loaded.


INFO:Global:Solution Data Correctly Loaded.


In [17]:
# For Freq==5.0
all_cap_data = cap_data[5.0]
all_cap_data

[5.0,
                          layer_1_datatype_0_plane layer_2_datatype_0_plane  \
 layer_1_datatype_0_plane                 0.234752                 0.170731   
 layer_2_datatype_0_plane                 0.170731                 0.329638   
 pad_bot_Q1                               0.021963                  0.02771   
 pad_top_Q1                               0.021657                 0.026484   
 readout_connector_pad_Q1                 0.002911                 0.031008   
 
                          pad_bot_Q1 pad_top_Q1 readout_connector_pad_Q1  
 layer_1_datatype_0_plane   0.021963   0.021657                 0.002911  
 layer_2_datatype_0_plane    0.02771   0.026484                 0.031008  
 pad_bot_Q1                 0.077165   0.025903                 0.001589  
 pad_top_Q1                 0.025903   0.088799                 0.014755  
 readout_connector_pad_Q1   0.001589   0.014755                 0.058203  ,
                          layer_1_datatype_0_plane layer_2_datatype

In [18]:
# For 1st pass
pass_1 = cap_data[5.0][1]
pass_1

,layer_1_datatype_0_plane,layer_2_datatype_0_plane,pad_bot_Q1,pad_top_Q1,readout_connector_pad_Q1
layer_1_datatype_0_plane,0.234752,0.170731,0.021963,0.021657,0.002911
layer_2_datatype_0_plane,0.170731,0.329638,0.02771,0.026484,0.031008
pad_bot_Q1,0.021963,0.02771,0.077165,0.025903,0.001589
pad_top_Q1,0.021657,0.026484,0.025903,0.088799,0.014755
readout_connector_pad_Q1,0.002911,0.031008,0.001589,0.014755,0.058203


In [19]:
# For 2nd pass
pass_2 = cap_data[5.0][2]
pass_2

,layer_1_datatype_0_plane,layer_2_datatype_0_plane,pad_bot_Q1,pad_top_Q1,readout_connector_pad_Q1
layer_1_datatype_0_plane,0.244783,0.188514,0.016333,0.014276,0.002106
layer_2_datatype_0_plane,0.188514,0.33466,0.028331,0.025276,0.033945
pad_bot_Q1,0.016333,0.028331,0.07317,0.026615,0.001891
pad_top_Q1,0.014276,0.025276,0.026615,0.083429,0.017263
readout_connector_pad_Q1,0.002106,0.033945,0.001891,0.017263,0.062585


In [20]:
# For 3rd pass
pass_3 = cap_data[5.0][3]
pass_3

,layer_1_datatype_0_plane,layer_2_datatype_0_plane,pad_bot_Q1,pad_top_Q1,readout_connector_pad_Q1
layer_1_datatype_0_plane,0.30142,0.2461,0.011354,0.010767,0.004714
layer_2_datatype_0_plane,0.2461,0.358678,0.034068,0.029999,0.036993
pad_bot_Q1,0.011354,0.034068,0.075927,0.026413,0.001947
pad_top_Q1,0.010767,0.029999,0.026413,0.087053,0.018457
readout_connector_pad_Q1,0.004714,0.036993,0.001947,0.018457,0.065863


In [ ]:
# 4th pass does not exist, since we limited passes to three passes.
#pass_4 = cap_data[5.0][4]
#pass_4

In [ ]:
# Will disconnect Metal from Ansys, and leave Ansys open.
#multi_q3d.close()

# # If the python script closes, the OS will most likely close Ansys.

# Will close Ansys.
#multi_q3d.force_exit_ansys()